# Brain teaser

[Original Github REPO](https://github.com/1171-jpg/BrainTeaser)


### Upute za pokretanje

1) Pokrenuti donju liniju i viditi jel se sve moze importati
1) Skinuti sve sto se ne moze sa `!pip install` unutar notebooka
(pip bi trebao raditi i s condom ako je dobro namjesteno)
    - Skinuti pytorch sa stranice [Pytorch](https://pytorch.org/get-started/locally/) - odaberite CUDA 11.8
    ako imate graficku. Meni je bilo:
    `> !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118`
    - Ako nemate CUDA, notebook ce biti sporiji - dovoljan za inference, 
    ali nedovoljan za fine-tuning (sreca pa to niti ne radimo, bilo bi presporo 
    i s grafickom)
    - Skinuti huggingface transformers library:
    `> pip install 'transformers[torch]'`
1) Moze se sve importati na Kaggle, ovo mozda malo kasnije

In [6]:
import numpy as np
import json
from datasets import load_dataset

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import torch
from torch.nn.functional import normalize
from transformers import AutoModel, AutoTokenizer

from tqdm.auto import tqdm
import numpy as np

from utils import getMetric, getSeperateResult, getResultdata

### Dataset

- Skinuti s [linka](https://drive.google.com/drive/u/0/folders/1kiFXp5fqpf8--NQJJAlBIBpfSaXTk1UY)

- Staviti u folder `/data`

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

### Load the model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM
# model_path = 'google/flan-t5-xxl'
# model_path = 'google/flan-t5-xl'
model_path = 'google/flan-t5-large'
# model_path = 'mistralai/Mistral-7B-Instruct-v0.2'
# model_path = 'microsoft/phi-2'

tokenizer = AutoTokenizer.from_pretrained(model_path)

if model_path.find('flan') >= 0:
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
else:
    model = AutoModelForCausalLM.from_pretrained(model_path)

In [9]:
try:
    model = model.to(device)
except Exception as e:
    print(e)
    print("Changing device to cpu")
    device = 'cpu'

In [11]:
# few_shot_examples = list(np.load("./{dataset_path}/data/demonstration.npy",allow_pickle=True))
sentence_data_path = "data/SP-train.npy"
wordplay_data_list = "data/WP-train.npy"
sentence_data_list = list(np.load(sentence_data_path,allow_pickle=True))
wordplay_data_list = list(np.load(wordplay_data_list,allow_pickle=True))

In [12]:
test_data_list = sentence_data_list + wordplay_data_list
print(f"Dataset length {len(test_data_list)}")

Dataset length 903


In [30]:
description = "The following question is a brainteaser. Choose the answer using elimination method. However, eliminate all the choices that are obvious from the question and respond with the least likely one."

format = \
"""
Question: {}

What is the correct answer to the question from the following choices?
(A) {}
(B) {}
(C) {}
(D) {}
"""

def get_sample_demo(sample):
    sample_demo = format.format(sample['question'],sample['choice_list'][0],sample['choice_list'][1],sample['choice_list'][2],sample['choice_list'][3])
    
    return sample_demo

In [31]:
print(get_sample_demo(test_data_list[0]))


The following question is a brainteaser. Choose the answer using elimination method. However, eliminate all the choices that are obvious from the question and respond with the least likely one.

Question: Mr. and Mrs. Mustard have six daughters and each daughter has one brother. But there are only 9 people in the family, how is that possible?

What is the correct answer to the question from the following choices?
(A) Some daughters get married and have their own family.
(B) Each daughter shares the same brother.
(C) Some brothers were not loved by family and moved away.
(D) None of above.



In [32]:
from collections import defaultdict

for sample in tqdm(test_data_list):
    inputs = tokenizer.encode(get_sample_demo(sample), return_tensors="pt").to(device)
    outputs = model.generate(inputs, max_new_tokens=10000)
    sample['response'] = tokenizer.decode(outputs[0])

100%|█████████████████████████████████████████████████████████████████████████████████| 903/903 [01:25<00:00, 10.57it/s]


In [38]:
for index,item in enumerate(test_data_list):
    try:
        item['predict'] = item['response'].split('(')[1].strip().split(')')[0].strip()
    except:
        try:
            item['predict'] = item['response'].split('> ')[1].strip().split('</')[0].strip()
        except:
            item['predict'] = None
            print(index)

In [46]:
word_play,sentence_play = getResultdata(test_data_list)
final_result = getSeperateResult(word_play,sentence_play)

#########Wordplay##########
over_all accuracy 0.19949494949494948
single_original_accuracy 0.18181818181818182
single_semantic_accuracy 0.1893939393939394
single_context_accuracy 0.22727272727272727
sr_accuracy 0.13636363636363635
cr_accuracy 0.09090909090909091
#########Sentence##########
over_all accuracy 0.14990138067061143
single_original_accuracy 0.14792899408284024
single_semantic_accuracy 0.13609467455621302
single_context_accuracy 0.16568047337278108
sr_accuracy 0.08284023668639054
cr_accuracy 0.04142011834319527
#########All data##########
over_all accuracy 0.1716500553709856
single_original_accuracy 0.16279069767441862
single_semantic_accuracy 0.15946843853820597
single_context_accuracy 0.19269102990033224
sr_accuracy 0.10631229235880399
cr_accuracy 0.06312292358803986


In [55]:
final_result

{'wordplay': {'over_all accuracy': 0.19949494949494948,
  'original_accuracy': 0.18181818181818182,
  'semantic_accuracy': 0.1893939393939394,
  'context_accuracy': 0.22727272727272727,
  'ori_sema': 0.13636363636363635,
  'ori_sema_cont': 0.09090909090909091},
 'sentence': {'over_all accuracy': 0.14990138067061143,
  'original_accuracy': 0.14792899408284024,
  'semantic_accuracy': 0.13609467455621302,
  'context_accuracy': 0.16568047337278108,
  'ori_sema': 0.08284023668639054,
  'ori_sema_cont': 0.04142011834319527},
 'all': {'over_all accuracy': 0.1716500553709856,
  'original_accuracy': 0.16279069767441862,
  'semantic_accuracy': 0.15946843853820597,
  'context_accuracy': 0.19269102990033224,
  'ori_sema': 0.10631229235880399,
  'ori_sema_cont': 0.06312292358803986}}

In [56]:
model_name = model_path.split('/')[-1]
with open(f'results/{model_name}-least-likely.txt', 'w') as file:
    file.write(model_name)
    file.write(json.dumps(final_result, indent=4))